In [1]:
using Random, Distributions, DataFrames, GLM

In [2]:
α = 1.78
β = 1
Q = 100 # Liczba państw

100

In [3]:
p = rand(Uniform(.1,.75), Q)

N = reduce(vcat, rand.(Binomial.(rand(DiscreteUniform(100, 1000), Q), .1), 1))
M = reduce(vcat, rand.(Poisson.(N .^ α), 1))

m = reduce(vcat, rand.(Poisson.(M .* p), 1))
n = reduce(vcat, rand.(Binomial.(N, p), 1))

100-element Vector{Int64}:
 30
  9
  9
 12
 13
 16
 10
  7
  5
 10
 17
 10
 14
  ⋮
 36
 54
 11
 16
 26
 17
  7
 65
 20
  5
 31
 24

In [4]:
df = DataFrame(
    y = log.(m ./ N),
    x1 = log.(N),
    x2 = log.(n ./ N)
)

Row,y,x1,x2
,Float64,Float64,Float64
1,2.31598,4.48864,-1.08744
2,1.63142,2.89037,-0.693147
3,2.00092,3.8712,-1.67398
4,1.67765,2.83321,-0.348307
5,1.24479,4.27667,-1.71172
6,2.18566,3.3673,-0.594707
7,1.89184,2.94444,-0.641854
8,0.670158,3.09104,-1.14513
9,1.14072,4.12713,-2.5177


In [5]:
df1 = DataFrame(
    y = log.(m),
    x1 = log.(N),
    x2 = log.(n ./ N)
)
lm(@formula(y ~ x1 + x2 + 0), df1)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

y ~ 0 + x1 + x2

Coefficients:
───────────────────────────────────────────────────────────────
      Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────
x1  1.72176   0.0139874  123.09    <1e-99   1.694      1.74951
x2  0.76965   0.0409743   18.78    <1e-33   0.688338   0.850963
───────────────────────────────────────────────────────────────

In [6]:
ols = lm(@formula(y ~ x1 + x2 + 0), df)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

y ~ 0 + x1 + x2

Coefficients:
───────────────────────────────────────────────────────────────
       Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────
x1  0.721757   0.0139874  51.60    <1e-72   0.693999   0.749515
x2  0.76965    0.0409743  18.78    <1e-33   0.688338   0.850963
───────────────────────────────────────────────────────────────

In [7]:
α̂, β̂ = coef(ols) + [1;0]

2-element Vector{Float64}:
 1.7217569537235904
 0.7696504022456492

In [8]:
ξ̂ = N .^ α̂

100-element Vector{Float64}:
 2271.8298392112274
  144.96867331908734
  784.6731485425602
  131.3814306107747
 1577.157774452257
  329.5292136384435
  159.11198893116142
  204.7979407789474
 1219.1660353542293
  433.345683428168
 1691.9977021537534
  273.04875534347985
  455.522561964361
    ⋮
 1023.1867132629354
 2315.9577371627906
 1890.8558375337666
 1152.2435351416655
  548.8157305934928
 1429.3380767192325
  237.89663716465995
 2141.5891622854565
 2315.9577371627906
   94.04894220615299
 1577.157774452257
  573.268304067692

In [9]:
@show [sum(M), sum(N .^ α), sum(ξ̂)]

[sum(M), sum(N .^ α), sum(ξ̂)] = [138385.0, 138960.2763015329, 108618.01925650507]


3-element Vector{Float64}:
 138385.0
 138960.2763015329
 108618.01925650507